# Commodity prices and Markov chains

<div align="right"><button><a href="https://colab.research.google.com/github/QuantEcon/workshop.africa-july2023/blob/main/day-08/exercise_set_8_with_solution.ipynb"><img src="" heght="10px"/><img
  src="https://colab.research.google.com/assets/colab-badge.svg"
  alt="open with Colab" width="100px"/></a></button></div>


#### Written for the QuantEcon Africa Workshop (July 2023)
#### Author: [Shu Hu](https://shu-hu.com/)

In [ ]:
!pip install quantecon

In [ ]:
import numpy as np
import quantecon as qe
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.optimize import minimize_scalar, brentq
from scipy.stats import norm, pareto, beta

### Exercise 1 (Commodity prices I)

Read the lecture [Commodity Prices](https://intro.quantecon.org/commod_price.html) before you attempt the following exercises.

**Exercise 1.1**

Keep all the other code the same as section [Code](https://intro.quantecon.org/commod_price.html#code), 

except that the shock $Z$ is set to
$$
    Z = a + c \cdot \xi
$$
where $\xi \sim N (0, 1)$.

In this exercise:

- Compute the the approximation of $p^*$ and 
- plot the approximation along with the inverse demand curve $P$.

**Solution**

In [ ]:
α, a, c = 0.8, 1.0, 2.0
mc_draw_size = 250
gridsize = 150
grid_max = 35
grid = np.linspace(a, grid_max, gridsize)

Z = a + norm.rvs(size=mc_draw_size) * c    # Shock observations
D = P = lambda x: 1.0 / x
tol = 1e-4


def T(p_array):

    new_p = np.empty_like(p_array)

    # Interpolate to obtain p as a function.
    p = interp1d(grid,
                 p_array,
                 fill_value=(p_array[0], p_array[-1]),
                 bounds_error=False)

    # Update
    for i, x in enumerate(grid):

        h = lambda q: q - max(α * np.mean(p(α * (x - D(q)) + Z)), P(x))
        new_p[i] = brentq(h, 1e-8, 100)

    return new_p


fig, ax = plt.subplots()

price = P(grid)
ax.plot(grid, price, alpha=0.5, lw=1, label="inverse demand curve")
error = tol + 1
while error > tol:
    new_price = T(price)
    error = max(np.abs(new_price - price))
    price = new_price

ax.plot(grid, price, 'k-', alpha=0.5, lw=2, label=r'$p^*$')
ax.legend()
ax.set_xlabel('$x$', fontsize=12)

plt.show()

**Exercise 1.2**

Recall that you have learned the lecture [Heavy-Tailed Distributions](https://intro.quantecon.org/heavy_tails.html).

Now do the same job as in Exercise 1.1 except that the shock $Z$ is set to
$$
    Z = a + c \cdot \chi
$$
where $\chi$ follows a Pareto distribution with a tail exponent of $b=1.5$.

**Solution**

In [ ]:
α, a, c = 0.8, 1.0, 2.0
b = 1.5 # set the Pareto tail index
mc_draw_size = 250
gridsize = 150
grid_max = 35
grid = np.linspace(a, grid_max, gridsize)

Z = a + pareto.rvs(b, size=mc_draw_size) * c    # Shock observations
D = P = lambda x: 1.0 / x
tol = 1e-4


def T(p_array):

    new_p = np.empty_like(p_array)

    # Interpolate to obtain p as a function.
    p = interp1d(grid,
                 p_array,
                 fill_value=(p_array[0], p_array[-1]),
                 bounds_error=False)

    # Update
    for i, x in enumerate(grid):

        h = lambda q: q - max(α * np.mean(p(α * (x - D(q)) + Z)), P(x))
        new_p[i] = brentq(h, 1e-8, 100)

    return new_p


fig, ax = plt.subplots()

price = P(grid)
ax.plot(grid, price, alpha=0.5, lw=1, label="inverse demand curve")
error = tol + 1
while error > tol:
    new_price = T(price)
    error = max(np.abs(new_price - price))
    price = new_price

ax.plot(grid, price, 'k-', alpha=0.5, lw=2, label=r'$p^*$')
ax.legend()
ax.set_xlabel('$x$', fontsize=12)

plt.show()

**Exercise 1.3**

Read section [Code](https://intro.quantecon.org/commod_price.html#code).

Using the approximation of $p^*$ from Exercise 1.2, simulate a time series of prices with length $T=100$.

**Solution**

In [ ]:
T = 100

# Turn the price array into a price function
p_star = interp1d(grid,
                  price,
                  fill_value=(price[0], price[-1]),
                  bounds_error=False)

def carry_over(x):
    return α * (x - D(p_star(x)))

def generate_cp_ts(init=1, n=T):
    X = np.empty(n)
    X[0] = init
    for t in range(n-1):
            Z = a + c * pareto.rvs(b)
            X[t+1] = carry_over(X[t]) + Z
    return p_star(X)

fig, ax = plt.subplots()
ax.plot(generate_cp_ts(), label="price")
ax.set_xlabel("time")
ax.legend()
plt.show()

### Exercise 2 (Markov chains)

Read the 
- lecture [Markov Chains: Basic Concepts](https://intro.quantecon.org/markov_chains_I.html) and 
- the [corresponding methods in the QuantEcon package](https://quanteconpy.readthedocs.io/en/latest/markov/core.html) 

before you attempt the following exercises.

**Exercise 2.1**

Using a method from the [QuantEcon package](https://quanteconpy.readthedocs.io/en/latest/markov/core.html), define a Markov chain object, called ``mc``, characterised by a stochastic matrix ``P`` and state values ``state_values``

In [ ]:
P = [[0.1, 0.3, 0.2, 0.4],
     [0.1, 0.4, 0.1, 0.4], 
     [0.3, 0.2, 0.3, 0.2], 
     [0.2, 0.1, 0.2, 0.5]]

state_values = ["recession", "slump", "recovery", "boom"]

**Solution**

In [ ]:
T = 5

In [ ]:
mc = qe.MarkovChain(P, state_values=state_values)

**Exercise 2.2**

Simulate a Markov chain with length of ``T=5`` using the Markov chain defined from Exercise 2.1 with an initial value ``initial_value``.

In [ ]:
initial_value = "recession"

**Solution**

In [ ]:
mc.simulate(ts_length=T, init=initial_value)

**Exercise 2.3**

Calculate the stationary distribution(s), called ``ψ_star``, for the Markov chain defined from Exercise 2.1.

**Solution**

In [ ]:
ψ_star = mc.stationary_distributions[0]
ψ_star

**Exercise 2.4**

With the Markov chain defined from Exercise 2.1., compute the marignal distribution $\psi_t = \psi_0 P^t$ with ``t=100`` and an initial distribution ``ψ_0``.

In [ ]:
ψ_0 = [0.1, 0.4, 0.3, 0.2]

Compare $\psi_t$ with the ``ψ_star`` from Exercise 2.3.

**Solution**

In [ ]:
t = 100
P_power = np.linalg.matrix_power(P, t)
ψ = ψ_0 @ P_power
ψ

Read
- lecture [Markov Chains: Irreducibility and Ergodicity](https://intro.quantecon.org/markov_chains_II.html)


before you attempt the following exercises.

**Exercise 2.5**

With the Markov chain defined from Exercise 2.1., check whether its stochastic matrix is irreducible.

**Solution**

In [ ]:
mc.is_irreducible

**Exercise 2.6**

With the Markov chain defined from Exercise 2.1.,
- simulate a path of length ``T = 100_000`` for it,
- calculate the fraction of time spent on each state values, called ``p_hats`` and
- compare it with the stationary distribution we computed from Exercise 2.3.

**Solution**

In [ ]:
T = 100_000
X = mc.simulate(ts_length=T)

In [ ]:
p_hats = []
for x0 in state_values:
    p_hat = (X == x0).cumsum() / (1 + np.arange(T, dtype=float))
    p_hats.append(p_hat[-1])

In [ ]:
p_hats

In [ ]:
ψ_star

### Exercise 3 (Commodity prices II)

Read the lecture [Commodity Prices](https://intro.quantecon.org/commod_price.html) before you attempt the following exercises.

**Exercise 3.1**

Keep all the other code the same as Exercuse 1.1, 

except that the inverse demand $P(x)$ is set to
$$
    P (x) = d + e \cdot x
$$
where $d, e = 100, -1$ and $x \in S$.

In this exercise:

- Compute the approximation of $p^*$ and
- plot the approximation along with the inverse demand curve $P$.

**Solution**

In [ ]:
α, a, c = 0.8, 1.0, 2.0
d, e = 100, -1
mc_draw_size = 250
gridsize = 150
grid_max = 35
grid = np.linspace(a, grid_max, gridsize)

Z = a + norm.rvs(size=mc_draw_size) * c    # Shock observations
P = lambda x: d +  e * x
D = lambda x: (x - d)/e
tol = 1e-4


def T(p_array):

    new_p = np.empty_like(p_array)

    # Interpolate to obtain p as a function.
    p = interp1d(grid,
                 p_array,
                 fill_value=(p_array[0], p_array[-1]),
                 bounds_error=False)

    # Update
    for i, x in enumerate(grid):

        h = lambda q: q - max(α * np.mean(p(α * (x - D(q)) + Z)), P(x))
        new_p[i] = brentq(h, 1e-8, 100)

    return new_p


fig, ax = plt.subplots()

price = P(grid)
ax.plot(grid, price, alpha=0.5, lw=1, label="inverse demand curve")
error = tol + 1
while error > tol:
    new_price = T(price)
    error = max(np.abs(new_price - price))
    price = new_price

ax.plot(grid, price, 'k-', alpha=0.5, lw=2, label=r'$p^*$')
ax.legend()
ax.set_xlabel('$x$', fontsize=12)

plt.show()

**Exercise 3.2**

In this exercise, do the same job as Exercuse 3.1, except that the inverse demand $P(x)$ is set to
$$
    P (x) = x^{-\rho}
$$
where $\rho = 1$ and $x \in S$.

**Solution**

In [ ]:
α, a, c = 0.8, 1.0, 2.0
ρ = 1
mc_draw_size = 250
gridsize = 150
grid_max = 35
grid = np.linspace(a, grid_max, gridsize)

Z = a + norm.rvs(size=mc_draw_size) * c    # Shock observations
P = lambda x: x**(-ρ)
D = lambda x: np.exp(- np.log(x) / ρ)
tol = 1e-4


def T(p_array):

    new_p = np.empty_like(p_array)

    # Interpolate to obtain p as a function.
    p = interp1d(grid,
                 p_array,
                 fill_value=(p_array[0], p_array[-1]),
                 bounds_error=False)

    # Update
    for i, x in enumerate(grid):

        h = lambda q: q - max(α * np.mean(p(α * (x - D(q)) + Z)), P(x))
        new_p[i] = brentq(h, 1e-8, 100)

    return new_p


fig, ax = plt.subplots()

price = P(grid)
ax.plot(grid, price, alpha=0.5, lw=1, label="inverse demand curve")
error = tol + 1
while error > tol:
    new_price = T(price)
    error = max(np.abs(new_price - price))
    price = new_price

ax.plot(grid, price, 'k-', alpha=0.5, lw=2, label=r'$p^*$')
ax.legend()
ax.set_xlabel('$x$', fontsize=12)

plt.show()